# Azure AI Document Intelligence 契約書モデルの分析

Document Intelligence 契約書モデルでは、高性能の光学式文字認識 (OCR) 機能を使用して、重要な契約書エンティティの選択したグループから主要なフィールドと明細を分析および抽出します。電話でキャプチャされた画像、スキャンされたドキュメント、デジタル PDF など、さまざまな形式や品質の契約書を使用できます。API は、関係者、管轄区域、契約書 ID、役職などの重要な情報を抽出し、構造化された JSON データ表現を返して、ドキュメントテキストを分析します。このモデルでは現在、英語のドキュメント形式がサポートされています。

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-contract?view=doc-intel-4.0.0#field-extraction

In [ ]:
!pip install azure-ai-documentintelligence --upgrade

In [ ]:
import azure.ai.documentintelligence
print("Azure Document Intelligence version: ", azure.ai.documentintelligence.__version__)

In [ ]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, AnalyzeResult

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
endpoint = "<your-endpoint>"
key = "<your-key>"

document_intelligence_client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))


## URL から呼ぶ方法
契約書モデルのモデル名は `prebuilt-contract` です。[対応言語](https://learn.microsoft.com/azure/ai-services/document-intelligence/language-support-prebuilt?view=doc-intel-4.0.0&tabs=languages%2Cthermal#contract)でないドキュメントの場合は、`prebuilt-layout` をお使いください。

In [ ]:
docUrl = "https://documentintelligence.ai.azure.com/documents/samples/prebuilt/contract.pdf"

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-contract", AnalyzeDocumentRequest(url_source=docUrl)
)
result: AnalyzeResult = poller.result()

## ファイルを直接アップロードする方法

In [ ]:
path_to_sample_documents = "./contract.pdf"

with open(path_to_sample_documents, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-contract", analyze_request=f, content_type="application/octet-stream"
    )
result: AnalyzeResult = poller.result()


### Debug用コード

[jsonpickle](https://pypi.org/project/jsonpickle/) ライブラリを使用すると `AnalyzeResult` オブジェクト構造をそのまま JSON へ保存できます。Document Intelligence の分析データを一旦自分の手元に置いて細かく調べたい時に有用です。VSCode の [Json Editor](https://marketplace.visualstudio.com/items?itemName=nickdemayo.vscode-json-editor) 拡張が便利です。

In [ ]:
import jsonpickle
# Debug 用(AnalyzeResultオブジェクト構造をそのままJSONへ)
json_data = jsonpickle.encode(result)
with open('analyzed_data.json', "w", encoding='utf-8') as f:
    f.write(json_data)

# JSON からオブジェクト構造を復元
# f = open("analyzed_data.json")
# json_str = f.read()
# result = jsonpickle.decode(json_str)

## テキスト行の手書きスタイル
応答では、各テキスト行が手書きスタイルであるかどうかと、信頼度スコアが分類されます。


In [ ]:
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

## ページ
ページ コレクションは、サービス応答に表示される最初のオブジェクトです。レイアウトモデルでは、印刷および手書きのスタイルテキストが `lines` および `words` として抽出されます。このモデルでは、抽出された単語の境界 `polygon` 座標と `confidence` を出力します。

### 選択マーク
ドキュメントから選択マークも抽出されます。抽出された選択マークは、各ページの `pages` コレクション内に示されます。これには、境界 `polygon`、`confidence`、および選択 `state` (`selected/unselected`) が含まれます。関連するテキスト (抽出された場合) も、開始インデックス (`offset`) と `length` として含まれます。`length` はドキュメントのテキスト全体を含む最上位の `content` プロパティを参照します。

In [ ]:
for page in result.pages:
    print(f"----Analyzing layout from page #{page.page_number}----")
    print(f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}")

    if page.lines:
        for line_idx, line in enumerate(page.lines):
            print(f"...Line # {line_idx} {line.content}")

    if page.selection_marks:
        for selection_mark in page.selection_marks:
            print(
                f"Selection mark is '{selection_mark.state}' within bounding polygon "
                f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
            )

## テーブル
レイアウトモデルでは、JSON 出力の `tables` セクションにテーブルが抽出されます。抽出されるテーブル情報には、列と行の数、行の範囲、列の範囲が含まれます。境界ポリゴンのある各セルは、その領域が `columnHeader` として認識されているかどうかにかかわらず、情報と共に出力されます。このモデルでは、回転されるテーブルの抽出がサポートされています。各テーブル セルには、行と列のインデックスと境界ポリゴン座標が含まれています。セル テキストの場合、このモデルは開始インデックス (`offset`) を含む `span` 情報を出力します。

In [ ]:
if result.tables:
    for table_idx, table in enumerate(result.tables):
        print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")
        if table.bounding_regions:
            for region in table.bounding_regions:
                print(f"Table # {table_idx} location on page: {region.page_number} is {region.polygon}")
        for cell in table.cells:
            print(f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'")
            if cell.bounding_regions:
                for region in cell.bounding_regions:
                    print(
                        f"...content on page {region.page_number} is within bounding polygon '{region.polygon}'\n"
                    )
print("----------------------------------------")

## 契約書
以下のドキュメントで定義されている契約書モデル特有のフィールドを列挙します。

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-contract?view=doc-intel-4.0.0#field-extraction

In [ ]:
if result.documents:
    for idx, contract in enumerate(result.documents):
        print(f"--------Analysis of contract #{idx + 1}--------")
        print(f"Contract type: {contract.doc_type if contract.doc_type else 'N/A'}")

        if contract.fields:
            title = contract.fields.get("Title")
            if title:
                print(f"Title: {title.get('valueString')} has confidence: " f"{title.confidence}")

            contract_id = contract.fields.get("ContractId")
            if contract_id:
                print(f"Contract ID: {contract_id.get('valueString')} has confidence: {contract_id.confidence}")

            parties = contract.fields.get("Parties")
            if parties:
                print("Parties involved:")
                for idx, item in enumerate(parties.get("valueArray")):
                  print(f"...Item #{idx + 1}")
                  item_address = item.get("valueObject").get("Address")
                  if item_address:
                      print(f"......Address: {item_address.get('content')} ")
                      if 'valueAddress' in item_address:
                          value_address = item_address.get('valueAddress')
                          print("......Address Details:")
                          for key, value in value_address.items():
                              print(f"............{key}: {value}")

                  item_clause = item.get("valueObject").get("Clause")
                  if item_clause:
                      print(f"......Clause: {item_clause.get('valueString')} ")
                  item_name = item.get("valueObject").get("Name")
                  if item_name:
                      print(f"......Name: {item_name.get('valueString')} ")
                  item_referencename = item.get("valueObject").get("ReferenceName")
                  if item_referencename:
                      print(f"......ReferenceName: {item_referencename.get('valueString')} ")

            execution_date = contract.fields.get("ExecutionDate")
            if execution_date:
                print(f"Execution Date: {execution_date.get('content')} has confidence: {execution_date.confidence}")

            expiration_date = contract.fields.get("ExpirationDate")
            if expiration_date:
                print(f"Expiration Date: {expiration_date.get('content')} has confidence: {expiration_date.confidence}")

            renewal_date = contract.fields.get("RenewalDate")
            if renewal_date:
                print(f"Renewal Date: {renewal_date.get('content')} has confidence: {renewal_date.confidence}")

            jurisdictions = contract.fields.get("Jurisdictions")
            if jurisdictions:
                print("Jurisdictions involved:")
                for idx, item in enumerate(jurisdictions.get("valueArray")):
                  print(f"...Item #{idx + 1}")
                  item_clause = item.get("valueObject").get("Clause")
                  if item_clause:
                      print(f"......Clause: {item_clause.get('valueString')} ")
                  item_region = item.get("valueObject").get("Region")
                  if item_region:
                      print(f"......Region: {item_region.get('valueString')} ")


